<a href="https://colab.research.google.com/github/ShozenD/AIMirror/blob/master/lectures/day2/lecture7/gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/imperial.png" width="250" vspace="8"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/mlgh.png" width="220" hspace="50" vspace="5"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/ammi.png" width="190"/>

<font size="6">Modern Statistics and Machine Learning
for Population Health in Africa </font>

<font size="4">24th - 28th March 2025</font>

</center>

# Gaussian Process Regression with Stan

## Objectives
In this tutorial, you will learn about how to implement a Gaussian Process (GP) in Stan.

By the end of this tutorial,
1. You will have a better understanding of implementing custom functions in Stan;
2. You will improve your ability to translate mathematics into Stan code;
3. You will have a better understanding of GPs and how to implement them in probabilistic programming languages.

## Flow of the tutorial
1. Brief description of the dataset
2. Review of Gaussian Processes
3. How to implement a GP in Stan

In [ ]:
# Install CmdStanPy for Google Colab
!curl -O "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/scripts/utilities.py"
from utilities import custom_install_cmdstan, test_cmdstan_installation
custom_install_cmdstan()

In [ ]:
# Import required packages
import numpy as np
import pandas as pd

from cmdstanpy import CmdStanModel
import arviz as az

import matplotlib.pyplot as plt
import seaborn as sns

# Aesthetics
sns.set_theme(style='whitegrid')
plt.rc('font', size=9)
plt.rc('axes', titlesize=10)
plt.rc('axes', labelsize=9)
plt.rc('xtick', labelsize=9)
plt.rc('ytick', labelsize=9)
plt.rc('legend', fontsize=9)

## The dataset
For this tutorial, we will use the `nile` dataset available via the `statsmodels` library. The dataset consists of annual flow measurements of the Nile River at Aswan from 1871 to 1970.

In [ ]:
from statsmodels.datasets import nile
nile = nile.load_pandas().data

# Plot the data
fig, ax = plt.subplots()
nile.plot(x='year', y='volume', ax=ax)
ax.set_title('Nile River Volume')
ax.set_xlabel('Year')
ax.set_ylabel('Volume (10^8 m^3)')
ax.set_xlim(1871, 1970)
plt.show()

### Data preprocessing
To make things easier for the model, we will normalise the year data to be between 0 and 1 and standardise the flow data to have a mean of 0 and a standard deviation of 1.

In [ ]:
volume = nile['volume'].values
year = nile['year'].values

# Standardise year
year_mean = year.mean()
year_std = year.std()
x = (year - year_mean) / year_std

# Standardise volume
volume_mean = volume.mean()
volume_std = volume.std()
y = (volume - volume_mean) / volume_std

print(x.shape, y.shape)

## Review of Gaussian Process Regression

Let $\mathbf{y} = (y_1,\ldots,y_n)^\top$ be a vector of outcomes, in this case flow volume. Let $\mathbf{x} = (x_1,\ldots,x_n)^\top$ be a vector of inputs, in this case year. We will model the data as
$$
\begin{align*}
\mathbf{y} &= \alpha + f(\mathbf{x}) + \boldsymbol{\varepsilon}, \\
f(\mathbf{x}) &\sim GP(m(\mathbf{x}), k(\mathbf{x},\mathbf{x}')), \\
\boldsymbol{\varepsilon} &\sim N(0, \sigma_{\varepsilon}^2\mathbf{I}_n),
\end{align*}
$$
where $m(\mathbf{x})$ is the mean function, $k(\mathbf{x},\mathbf{x}')$ is the covariance function, and $\sigma_{\varepsilon}^2$ is the noise variance. For simplicity, we will assume that $m(\mathbf{x}) = 0$
$$
f(\mathbf{x}) \sim GP(0, k(\mathbf{x},\mathbf{x}')).
$$

Recall that for finite data, the zero mean GP prior is a multivariate Gaussian distribution with mean vector $\mathbf{0}$ and covariance matrix $\mathbf{K}$, where $K_{ij} = k(x_i,x_j)$.
$$
\mathbf{K} = \begin{pmatrix}
k(x_1,x_1) & \cdots & k(x_1,x_n) \\
\vdots & \ddots & \vdots \\
k(x_n,x_1) & \cdots & k(x_n,x_n)
\end{pmatrix}.
$$

The function $k$ is referred to as a covariance function or kernel function. Common choices for the kernel function include the squared exponential kernel, the Matern kernel, and the periodic kernel. As we shall see later, the choice of kernel characterises the smoothness of the GP. If, for example, we chose the squared exponential kernel, then $k$ has the form
$$
k(x,x') = \sigma^2\exp\left(-\frac{(x-x')^2}{2\ell^2}\right),
$$
where $\sigma^2$ is the marginal variance and $\ell$ is the lengthscale.

To sample from a GP, we first compute the covariance matrix $\mathbf{K}$ for a given set of inputs $\mathbf{x}$, kernel $k$, and hyperparameters. We then perform the Cholesky decomposition of $\mathbf{K}$ such that
$$
\mathbf{K} = \mathbf{L}\mathbf{L}^\top,
$$
where $\mathbf{L}$ is a lower triangular matrix. We then sample an auxiliary vector $\mathbf{z}$ from a standard multivariate normal distribution, i.e., $\mathbf{z} \sim N(\mathbf{0},\mathbf{I}_n)$. Finally, we compute the GP sample as
$$
\mathbf{f} = \mathbf{L}\mathbf{z}.
$$

## Implementing a GP in Stan

### The GP prior
To keep the code in the other blocks clean, we will define the GP prior in the `functions` block. We will write a custom function named `gp_se` that takes in the input points `x`, the marginal variance `alpha`, and the lengthscale `ell`, and a vector of auxiliary variables `z`. The function will return the GP sample `f`. The code will look something like this:

```stan
functions {
	vector gp_se(vector x, real sigma, real ell, vector z) {
		int n = rows(x);
		matrix[n,n] K;
		vector[n] f;
		matrix[n,n] L;

		// 1. Compute the covariance matrix (don't forget to add the nuggget term)

		// 2. Perform the Cholesky decomposition

		// 3. Compute the GP sample

		return f;
	}
}
```

Your task is to fill in the missing parts of the code. Refer to the [Stan Functions Reference](https://mc-stan.org/docs/functions-reference/index.html) on how to compute the covariance matrix, how to construct an identity matrix (for the nugget term), and on how to perform the Cholesky decomposition.

### The data block
In the data block, we will declare the number of data points `N`, input points `x` and the outcome `y`. Do you remember how to do this?

### The parameters block
In the parameters block, the hyperparameters of the GP, namely the marginal variance $\sigma$ and the lengthscale $\ell$. We also need to declare the intercept $\alpha$, auxiliary variables $\mathbf{z}$ and the noise variance $\sigma_\varepsilon$.

### Transformed parameters block
In the transformed parameters block, we will compute the GP sample `f` using the custom function `gp_se` that we defined earlier.

### The model block
In the model block, we specify the likelihood and the priors for each parameter and the auxiliary random variables. We will assume a normal likelihood for the data and inverse-gamma priors for the hyperparameters. We will also assume an inverse-gamma prior for the noise variance.
$$
\begin{align*}
\sigma &\sim \text{inv-Gamma}(1,5), \\
\ell &\sim \text{inv-Gamma}(1,5), \\
\sigma_{\varepsilon} &\sim \text{inv-Gamma}(1,5) \\
z_i &\sim N(0, 1).
\end{align*}
$$
In case you are wondering, below is a plot of the density of the inverse-gamma distribution with shape parameter 1 and scale parameter 5.

In [ ]:
from scipy import stats

# Parameters for the inverse-gamma distribution
alpha = 1  # shape parameter
beta = 5   # scale parameter

# Generate x values
x_vals = np.linspace(0.1, 20, 1000)  # avoiding x=0 as PDF is undefined there

# Calculate PDF
pdf = stats.invgamma.pdf(x_vals, alpha, scale=beta)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(x_vals, pdf, 'b-', lw=2)
plt.title('Inverse-Gamma Distribution (α=1, β=5)')
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.grid(True)
plt.xlim(0, 20)
plt.show()

### Generated quantities block
Finally, in the generated quantities block, we will compute the log likelihood for each data point (for model comparison) and the predicted values for the input points.

### Compiling and running the Stan program
Recall that to compile the Stan program, we use the `CmdStanModel` class from the `cmdstanpy` library. You will need to specify the path to your `.stan` file.

After compiling the model, you need to prepare the data as a dictionary and pass it to the `CmdStanModel` object's `sample` method.

Use the `sample` method to execute the inference algorithm. Run 4 chains with 500 warmup iterations and 1000 sampling iterations. Set the `adapt_delta` argument to 0.95 (for more stable sampling). Set a seed for replicability.

In [ ]:
import time

start_time = time.time()

# ==========
# Your code here
# ==========

end_time = time.time()
runtime = end_time - start_time
print(f"Runtime of the Stan model: {runtime} seconds")

How long did it take the inference algorithm to run?

### Posthoc model diagnostics
We should always check for issues in the sampling process and whether the MCMC chains have converged. We can use the `CmdStanMCMC` object's `diagnose` method to check for divergences, treedepth issues, and other potential problems.

Use the `CmdStanMCMC` object's `summary` method to get a summary of the posterior samples. Check if there are any parameters where $\hat{R} > 1.1$ and where the `ESS_bulk` is less than 1000.

### Posterior analysis

Use the `from_cmdstanpy` function in Arviz to convert the fit object into an inference data object.

Define a dictionary called `custom_summary_fns` that has 3 elements:
1. "median": A lambda function that calculates the median of a given sequence (50th percentile)
2. "q2.5": A lambda function that calculates the 2.5% percentile of a given sequence
3. "q97.5": A lambda function that calculates the 97.5% percentile of a given sequence

Pass this dictionary to the `stat_funcs` argument of the `summary` method to obtain the desired summary statistics for `mu`, the expected flow rate.

Plot the posterior distribution of `mu` agains the real data

### Posterior predictive checks
In a similar fashion to `mu`, calculate the median and the 95% credible intervals for the posterior predictive distribution. Plot this against the real data.

Calculate the proportion of data points within the 95% posterior credible interval.